<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Creating Virtual Machines with Python

## Azure Actions Covered

* Creating virtual machines


In this lecture, we're going to take a look at how to create virtual machines via the Python SDK.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). We'll add some new imports as well.

In [1]:
from azure.identity import AzureCliCredential
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.resource import ResourceManagementClient

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

In [2]:
credential = AzureCliCredential()
resource_client = ResourceManagementClient(credential, AZURE_SUBSCRIPTION_ID)

In [3]:
VNET_NAME = 'test-vnet'
SUBNET_NAME = 'test-subnet'

In [4]:
network_client = NetworkManagementClient(credential, AZURE_SUBSCRIPTION_ID)

In [5]:
poller = network_client.virtual_networks.begin_create_or_update(
    DEFAULT_RESOURCE_GROUP,
    VNET_NAME,
    {
        'location': DEFAULT_LOCATION,
        'address_space': {
            'address_prefixes': ['10.0.0.0/16'],
        }
    }
)

In [11]:
result = poller.result()

In [13]:
result.provisioning_state

'Succeeded'

In [14]:
poller = network_client.subnets.begin_create_or_update(
    DEFAULT_RESOURCE_GROUP,
    VNET_NAME,
    SUBNET_NAME,
    {'address_prefix': '10.0.0.0/24'}
)
subnet_result = poller.result()

In [15]:
subnet_result.provisioning_state

'Succeeded'

In [16]:
IP_NAME = 'vm-test-ip'
poller = network_client.public_ip_addresses.begin_create_or_update(
    DEFAULT_RESOURCE_GROUP,
    IP_NAME,
    {
        'location': DEFAULT_LOCATION,
        'sku': {'name': 'Standard'},
        'public_ip_allocation_method': 'Static',
        'public_ip_address_version': 'IPV4'
    }
)
ip_address_result = poller.result()

In [17]:
ip_address_result.name

'vm-test-ip'

In [19]:
ip_address_result.ip_address

'172.190.56.70'

In [22]:
NIC_NAME = 'test-nic'
poller = network_client.network_interfaces.begin_create_or_update(
    DEFAULT_RESOURCE_GROUP,
    NIC_NAME,
    {
        'location': DEFAULT_LOCATION,
        'ip_configurations': [
            {
                'name': 'test-ip-config',
                'subnet': {'id': subnet_result.id},
                'public_ip_address': {'id': ip_address_result.id}
            }
        ]
    }
)
nic_result = poller.result()

In [23]:
nic_result.name

'test-nic'

In [26]:
compute_client = ComputeManagementClient(credential, AZURE_SUBSCRIPTION_ID)

In [29]:
poller = compute_client.virtual_machines.begin_create_or_update(
    DEFAULT_RESOURCE_GROUP,
    'test-vm',
    {
        'location': DEFAULT_LOCATION,
        'storage_profile': {
            'image_reference': {
                'publisher': 'Canonical',
                'offer': 'UbuntuServer',
                'sku': '18.04-LTS',
                'version': 'latest'
            }
        },
        'hardware_profile': {'vm_size': 'Standard_A2_v2'},
        'os_profile': {
            'computer_name': 'test-vm',
            'admin_username': 'testuser',
            'admin_password': 'testpassword123!'
        },
        'network_profile': {
            'network_interfaces': [
                {'id': nic_result.id}
            ]
        }
    }
)
vm_result = poller.result()

In [30]:
vm_result.name

'test-vm'

In [32]:
for vm in compute_client.virtual_machines.list_all():
    print(vm.name)

test-vm
